In [123]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

# replace variables here.
url = "https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Toronto%2C%20Ontario%2C%20Canada&geoId=100025096&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
no_of_jobs = 25


# this will open up new window with the url provided above 

driver = webdriver.Chrome('C:\\Users\\Zack Pan\\Downloads\\chromedriver.exe')  
#executable_path = {'executable_path':'C:\\Users\\Zack Pan\\Downloads\\chromedriver.exe'}
#browser = Browser('chrome', **executable_path)

#driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)



In [124]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/20): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    
    sleep(5)
    
    
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))



You are scraping information about 21 jobs.


In [125]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []
urls =  []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)
 
    # url
    url = job.find("a", class_="result-benefits__text")
    url = job.findAll('a', {'class': 'result-card__full-card-link'})[0]['href']
    urls.append(url)
    
# to check if we have all information
print((job_id))
print((post_date))
print((company_name))
print((post_title))
print((urls))


['2427800619', '2538758168', '2548368108', '2542384180', '2542029589', '2514377787', '2543962446', '2550007033', '2446997790', '2542237314', '2503829376', '2513149728', '2542171294', '2504001553', '2537837550', '2504490557', '2536796600', '2534729047', '2428673628', '2529486710', '2526096308']
['2021-02-01', '2021-05-08', '2021-04-28', '2021-05-14', '2021-05-10', '2021-04-20', '2021-05-11', '2021-05-14', '2021-02-08', '2021-05-10', '2021-04-30', '2021-05-01', '2021-05-14', '2021-04-14', '2021-05-15', '2021-05-15', '2021-05-15', '2021-05-13', '2021-04-20', '2021-05-01', '2021-05-10']
['Yelp Graphic', 'Bonsai Graphic', 'Borrowell Graphic', 'Jungle Media Graphic', 'ContentFly Graphic', 'Canada Goose Graphic', 'Postmedia Network Inc. Graphic', 'Havas Canada | Havas Media Graphic', 'Brookfield Asset Management Graphic', 'CST Consultants Inc. Graphic', 'dentsu international Graphic', 'OMERS Graphic', 'FLEETCOR Graphic', 'Jerry Graphic', 'RBC Graphic', 'BAE Systems Applied Intelligence Graphi

In [126]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,


'url': urls
})


# cleaning description column
#job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        21 non-null     object
 1   Date          21 non-null     object
 2   Company Name  21 non-null     object
 3   Post          21 non-null     object
 4   Location      21 non-null     object
 5   url           21 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB
None


,Job ID,Date,Company Name,Post,Location,url
0,2427800619,2021-02-01,Yelp Graphic,Data Science Analyst (Remote),"Toronto, Ontario, Canada",https://ca.linkedin.com/jobs/view/data-science...
1,2538758168,2021-05-08,Bonsai Graphic,Associate Data Analyst,"Toronto, Ontario, Canada",https://ca.linkedin.com/jobs/view/associate-da...
2,2548368108,2021-04-28,Borrowell Graphic,Data Analyst,"Toronto, Ontario, Canada",https://ca.linkedin.com/jobs/view/data-analyst...
3,2542384180,2021-05-14,Jungle Media Graphic,Data Analyst,"Toronto, Ontario, Canada",https://ca.linkedin.com/jobs/view/data-analyst...
4,2542029589,2021-05-10,ContentFly Graphic,Data Analyst,"Toronto, Ontario, Canada",https://ca.linkedin.com/jobs/view/data-analyst...


In [87]:
job_data.to_csv('job_details.csv')